Based on https://bambinos.github.io/bambi/notebooks/wald_gamma_glm.html

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
az.style.use("arviz-darkgrid")
np.random.seed(1234)

In [ ]:
data = bmb.load_data("carclaims")
data.head()

In [ ]:
data = data[data["claimcst0"] > 0]

In [ ]:
data[data["claimcst0"] > 15000].shape[0]

In [ ]:
plt.hist(data[data["claimcst0"] <= 15000]["claimcst0"], bins=30)
plt.title("Distribution of claim amount")
plt.xlabel("Claim amount ($)");

In [ ]:
model_wald = bmb.Model(
    "claimcst0 ~ C(agecat) + gender + area",
    data.sample(n=200, replace=False),
    family="wald",
    link="log",
)
try:
    fitted_wald = model_wald.fit(
        tune=2000, target_accept=0.9, idata_kwargs={"log_likelihood": True}
    )
except EOFError:
    pass

In [ ]:
az.plot_trace(fitted_wald);

In [ ]:
az.summary(fitted_wald)

In [ ]:
model_gamma = bmb.Model(
    "claimcst0 ~ agecat + gender + area",
    data.sample(n=200, replace=False),
    family="gamma",
    link="log",
    categorical="agecat",
)
try:
    fitted_gamma = model_gamma.fit(
        tune=2000, target_accept=0.9, idata_kwargs={"log_likelihood": True}
    )
except EOFError:
    pass

In [ ]:
az.plot_trace(fitted_gamma);

In [ ]:
az.summary(fitted_gamma)

In [ ]:
models = {"wald": fitted_wald, "gamma": fitted_gamma}
df_compare = az.compare(models)
df_compare

In [ ]:
az.plot_compare(df_compare, insample_dev=False);